In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import openslide
import pandas as pd
import pickle

In [4]:
from fastai.callbacks.csv_logger import CSVLogger

In [5]:
from object_detection_fastai.helper.object_detection_helper import *
from object_detection_fastai.helper.wsi_loader import *
from object_detection_fastai.loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from object_detection_fastai.models.RetinaNet import RetinaNet
from object_detection_fastai.callbacks.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric, PascalVOCMetricByDistance

In [6]:
slides_train = ["2707_20 TB BAL Human neu.svs", 
                         "11480_19 humane BAL Berliner Blau.svs", 
                         "10080_19 humane BAL Berliner Blau-001.svs", 
                         "10052_19 humane BAL Berliner Blau-001.svs", 
                         "10120_19 humane BAL Berliner Blau.svs" 
                        ]

slides_val = ["10227_19 humane BAL Berliner Blau.svs", "2702_20 BB Human BAL-001.svs"]

In [7]:
grade_list = [0, 1]
annotations_path = Path("../../Statistics/SDATA_Final_Annotations.pkl")
annotations = pd.read_pickle(annotations_path)
annotations["image_name"] = [name.replace("tiff","svs") for name in annotations["image_name"]]
annotations = annotations[annotations["grade"].isin(grade_list)]
annotations_train = annotations[annotations["image_name"].isin(slides_train)]
annotations_val = annotations[annotations["image_name"].isin(slides_val)]
annotations_train.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version
139421,2219209,3622,251,human,2707_20 TB BAL Human neu.svs,TurnbullBlue,0,"{'x1': 11061, 'x2': 11239, 'y1': 15500, 'y2': ...",eee6bb47-8825-4de6-939a-92d35e07485b,1,False,1,SDATA,Inference
139422,2219210,3622,251,human,2707_20 TB BAL Human neu.svs,TurnbullBlue,0,"{'x1': 7575, 'x2': 7722, 'y1': 11417, 'y2': 11...",5aa682e3-1417-4104-b3ab-41d316662e4f,1,False,1,SDATA,Inference
139423,2219211,3622,251,human,2707_20 TB BAL Human neu.svs,TurnbullBlue,0,"{'x1': 11418, 'x2': 11583, 'y1': 13947, 'y2': ...",4ac20d34-177e-4e30-97a4-16a26372b39f,1,False,1,SDATA,Inference
139424,2219212,3622,251,human,2707_20 TB BAL Human neu.svs,TurnbullBlue,0,"{'x1': 19568, 'x2': 19753, 'y1': 20871, 'y2': ...",4364a598-cd6a-4c27-b09b-f4503e44bc61,1,False,1,SDATA,Inference
139425,2219213,3622,251,human,2707_20 TB BAL Human neu.svs,TurnbullBlue,0,"{'x1': 7516, 'x2': 7674, 'y1': 11404, 'y2': 11...",5e8f8297-6453-4c1f-8585-8462b016f9ab,1,False,1,SDATA,Inference


In [8]:
slides_path = Path("../../../Slides")
files = {slide.name: slide for slide in slides_path.rglob("*.svs")  if slide.name in slides_train + slides_val}
files

{'10052_19 humane BAL Berliner Blau-001.svs': PosixPath('../../../Slides/Human/10052_19 humane BAL Berliner Blau-001.svs'),
 '10080_19 humane BAL Berliner Blau-001.svs': PosixPath('../../../Slides/Human/10080_19 humane BAL Berliner Blau-001.svs'),
 '10227_19 humane BAL Berliner Blau.svs': PosixPath('../../../Slides/Human/10227_19 humane BAL Berliner Blau.svs'),
 '10835_19 humane BAL Berliner Blau.svs': PosixPath('../../../Slides/Human/10835_19 humane BAL Berliner Blau.svs'),
 '11480_19 humane BAL Berliner Blau.svs': PosixPath('../../../Slides/Human/11480_19 humane BAL Berliner Blau.svs'),
 '2702_20 BB Human BAL-001.svs': PosixPath('../../../Slides/Human/2702_20 BB Human BAL-001.svs'),
 '2707_20 TB BAL Human neu.svs': PosixPath('../../../Slides/Human/2707_20 TB BAL Human neu.svs')}

In [9]:
tfms = get_transforms(do_flip=True,
                      flip_vert=True,
                      #max_rotate=90,
                      max_lighting=0.0,
                      max_zoom=1.,
                      max_warp=0.0,
                      p_affine=0.5,
                      p_lighting=0.0,
                      #xtra_tfms=xtra_tfms,
                     )
tfms

([RandTransform(tfm=TfmCrop (crop_pad), kwargs={'row_pct': (0, 1), 'col_pct': (0, 1), 'padding_mode': 'reflection'}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True),
  RandTransform(tfm=TfmAffine (dihedral_affine), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True),
  RandTransform(tfm=TfmAffine (rotate), kwargs={'degrees': (-10.0, 10.0)}, p=0.5, resolved={}, do_run=True, is_random=True, use_on_y=True)],
 [RandTransform(tfm=TfmCrop (crop_pad), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True)])

In [10]:
size = 1024 
level = 0
bs = 16
train_images = 2500
val_images = 1500

In [11]:
def get_y_func(x):
    return x.y

In [12]:
anchors = create_anchors(sizes=[(32,32)], ratios=[1], scales=[0.6, 0.7, 0.9, 1.25, 1.5])

In [13]:
for i in range(len(slides_train)):
    
    torch.cuda.empty_cache()

    train_files = []
    val_files = []
    
    experiment_name = "HumanVsHuman-Ablation_{}".format(i)

    for image_name in slides_train[:i+1]:

        annotations = annotations_train[annotations_train["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]

        train_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))

    for image_name in annotations_val["image_name"].unique():

        annotations = annotations_val[annotations_val["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]

        val_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))
    
    train_files = list(np.random.choice(train_files, train_images))
    valid_files = list(np.random.choice(val_files, val_images))
    
    train =  ObjectItemListSlide(train_files, path=slides_path)
    valid = ObjectItemListSlide(valid_files, path=slides_path)
    item_list = ItemLists(slides_path, train, valid)
    lls = item_list.label_from_func(get_y_func, label_cls=SlideObjectCategoryList) #
    lls = lls.transform(tfms, tfm_y=True, size=size)
    data = lls.databunch(bs=bs, collate_fn=bb_pad_collate).normalize() #, num_workers=0
    
    crit = RetinaNetFocalLoss(anchors)
    encoder = create_body(models.resnet18, True, -2)
    model = RetinaNet(encoder, n_classes=data.train_ds.c, n_anchors=5, sizes=[32], chs=128, final_bias=-4., n_conv=3)
    
    voc = PascalVOCMetricByDistance(anchors, size, [str(i) for i in data.train_ds.y.classes[1:]], radius=40)
    learn = Learner(data, model, loss_func=crit, callback_fns=[BBMetrics, partial(CSVLogger, append=False, filename=experiment_name)], #BBMetrics, ShowGraph
                    metrics=[voc])

    learn.split([model.encoder[6], model.c5top5])
    learn.freeze_to(-2)

    
    learn.unfreeze()
    learn.fit_one_cycle(10, 1e-3)
    
    learn.destroy() 

epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.436168,0.368174,0.644195,0.105801,0.262373,0.562033,0.726356,32:35
1,0.237520,0.311702,0.799099,0.088706,0.222996,0.770783,0.827416,20:52
2,0.203626,0.233123,0.822653,0.071187,0.161936,0.752687,0.892618,25:34
3,0.177158,0.211993,0.847651,0.071896,0.140097,0.821751,0.873551,21:43
4,0.162890,0.188325,0.870640,0.060466,0.127859,0.828389,0.912891,26:21
5,0.139770,0.174559,0.872297,0.056700,0.117859,0.854948,0.889647,32:41
6,0.128402,0.165326,0.896982,0.058945,0.106381,0.869627,0.924338,25:10
7,0.114608,0.156791,0.894810,0.052044,0.104747,0.851874,0.937746,12:54
8,0.103210,0.168468,0.883778,0.052839,0.115629,0.837096,0.930461,13:09
9,0.095985,0.162671,0.894315,0.051096,0.111575,0.846992,0.941639,12:59


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.497708,0.349139,0.729216,0.115811,0.233328,0.667111,0.791321,23:34
1,0.227870,0.211041,0.868691,0.071870,0.139170,0.818101,0.919280,15:07
2,0.204096,0.177166,0.901226,0.062774,0.114392,0.881360,0.921092,16:31
3,0.187246,0.181762,0.905277,0.069613,0.112150,0.893918,0.916635,17:25
4,0.165330,0.234745,0.887378,0.068628,0.166117,0.878988,0.895769,14:08
5,0.151559,0.189762,0.849425,0.057614,0.132148,0.872327,0.826522,13:44
6,0.132711,0.132572,0.926766,0.050464,0.082108,0.924433,0.929099,12:34
7,0.121461,0.129785,0.922748,0.046231,0.083554,0.932693,0.912804,13:27
8,0.110912,0.118081,0.937108,0.045123,0.072958,0.931996,0.942221,12:02
9,0.104013,0.117860,0.939130,0.045327,0.072533,0.939108,0.939153,12:55


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.459955,0.338157,0.772108,0.119085,0.219072,0.739990,0.804227,22:50
1,0.234747,0.222075,0.880673,0.075976,0.146100,0.839100,0.922246,41:51
2,0.200038,0.266047,0.851694,0.082101,0.183946,0.841918,0.861470,13:42
3,0.173903,0.214065,0.863366,0.059626,0.154438,0.849542,0.877190,18:04
4,0.160943,0.192584,0.915312,0.077450,0.115134,0.923103,0.907520,14:26
5,0.142649,0.162630,0.911864,0.049003,0.113628,0.935692,0.888035,15:53
6,0.130594,0.164075,0.926558,0.068979,0.095096,0.912047,0.941070,16:35
7,0.114787,0.121109,0.935824,0.046791,0.074317,0.929173,0.942474,14:13
8,0.108151,0.116621,0.939655,0.043892,0.072729,0.928407,0.950902,13:39
9,0.104004,0.117238,0.940840,0.045790,0.071448,0.933811,0.947869,13:15


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.488425,0.389140,0.740258,0.112592,0.276548,0.687208,0.793309,15:13
1,0.318501,0.272658,0.815445,0.092115,0.180542,0.746364,0.884526,20:11
2,0.411726,0.471793,0.776549,0.117598,0.354195,0.737482,0.815615,24:54
3,0.235799,0.402397,0.757184,0.070260,0.332137,0.690063,0.824306,09:23
4,0.203417,0.230129,0.828061,0.072119,0.158009,0.769752,0.886370,16:42
5,0.186765,0.171379,0.901308,0.064694,0.106685,0.902211,0.900404,15:40
6,0.167914,0.158233,0.896531,0.052362,0.105871,0.863717,0.929345,13:17
7,0.147651,0.139585,0.918765,0.048536,0.091050,0.918816,0.918714,14:15
8,0.138401,0.132350,0.928313,0.047375,0.084975,0.909364,0.947262,14:05
9,0.132966,0.131091,0.926155,0.045570,0.085521,0.910981,0.941328,12:52


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.546438,0.455300,0.601595,0.144040,0.311259,0.398039,0.805150,19:27
1,0.332942,0.327924,0.718893,0.091014,0.236910,0.588553,0.849234,18:00
2,0.327047,0.264986,0.820093,0.094737,0.170250,0.808821,0.831364,15:53
3,0.256386,0.425017,0.604097,0.085642,0.339375,0.583072,0.625123,25:18
4,0.226383,0.237752,0.823297,0.067620,0.170133,0.851637,0.794956,11:49
5,0.218022,0.221596,0.852587,0.078289,0.143307,0.848511,0.856663,13:34
6,0.186605,0.174102,0.890507,0.056264,0.117838,0.890480,0.890534,12:44
7,0.164812,0.161155,0.905447,0.050440,0.110714,0.897438,0.913456,13:06
8,0.152129,0.157682,0.896363,0.049358,0.108324,0.878563,0.914162,13:08
9,0.145458,0.155727,0.903903,0.047606,0.108121,0.890507,0.917298,12:39


this Learner object self-destroyed - it still exists, but no longer usable
